In [4]:
import numpy as np
import pandas as pd
import pandas_profiling
import os
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

#from matplotlib import scatter_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from zipfile import ZipFile

## Create function to Read in Data

In [5]:
def read_in_CollegeScorecard(columns):
    """
    Read in columns from files from inside the zip file. 
    Also assign a year to each DataFrame.
    
    Parameters:
    -----------
    columns: list,
        columns is a list of strings matching the desired column headers
        
    Returns:
    --------
    sheets: dictionary,
        sheets is a dictionary of year:DataFrame pairs
    """
    
    zip_file = ZipFile('CollegeScorecard_Raw_Data.zip')
    
    sheets = {}
    for year in range(1996, 2018):
        acyear = str(year)+'_'+str(year+1)[-2:]
        sheets[year] = pd.read_csv(zip_file.open('CollegeScorecard_Raw_Data/MERGED'+acyear+'_PP.csv'), usecols=columns)
        sheets[year]['YEAR'] = year
        sheets[year]['YEAR'] = pd.to_datetime(sheets[year]['YEAR'], format='%Y')
    return sheets

In [6]:
columns = ['INSTNM', 'HIGHDEG', 'CONTROL', 'REGION', 'LOCALE', 'LO_INC_DEBT_N', 'MD_INC_DEBT_N', 'HI_INC_DEBT_N',
           'LOAN_EVER', 'PELL_EVER', 'PCTPELL', 'ICLEVEL', 'CURROPER', 'TUITFTE', 'CDR3', 'INEXPFTE']
sheets = read_in_CollegeScorecard(columns)

## Create concatenated df of all sheets

In [7]:
def concatenate_all_sheets(sheets):
    """
    Concatenates DataFrames in a dictionary of DataFrames.
    
    Parameters:
    -----------
    sheets: dictionary,
        key value pairs are year and DataFrame associated to that year
        
    Returns:
    --------
    full_df: DataFrame
    """
    for year, df in sheets.items():
        df['iyear'] = df['YEAR']
        if year==1996:
            full_df = df.set_index([df.index, 'iyear'])
        else:
            full_df = pd.concat([full_df, df.set_index([df.index, 'iyear'])])
    return full_df

In [8]:
full_df = concatenate_all_sheets(sheets)

## Back Fill huge DataFrame

In [15]:
def back_fill_from_year(df, year):
    fill_values = {}
    for name in df.INSTNM.unique():
        fill_values[name] = {'LOCALE': df.loc[(df.INSTNM==name)].LOCALE.values[0], 
                             'CURROPER': df.loc[(df.INSTNM==name)].CURROPER.values[0],
                             'CONTROL': df.loc[(df.INSTNM==name)].CONTROL.values[0]}
    
    for name in df.loc[df.YEAR!=year].INSTNM.unique():
        fill_values[name] = fill_values.get(name, {'LOCALE': np.NaN, 'CURROPER': np.NaN, 'CONTROL': np.NaN})
        
    copy_df = df.drop(['LOCALE', 'CURROPER', 'CONTROL'], axis=1)
    
    copy_df['LOCALE'] = df.INSTNM.map(lambda name: fill_values[name]['LOCALE'])
    copy_df['CURROPER'] = df.INSTNM.map(lambda name: fill_values[name]['CURROPER'])
    copy_df['CONTROL'] = df.INSTNM.map(lambda name: fill_values[name]['CONTROL'])
    
    return copy_df

In [16]:
df_2016_2017 = back_fill_from_year(full_df.loc[(full_df.YEAR==2016)|(full_df.YEAR==2017)], 2017)

## Clean full_df

In [ ]:
full_df.info()

In [ ]:
def quick_clean(df, columns=None):
    """
    drops rows with NaNs in the columns given. 
    If no columns are given, drops all rows with any nans.
    
    Parameters:
    -----------
    df: DataFrame
    
    columns: list
    
    Returns:
    --------
    df_nonan: dataframe
    """
    df_unsuppressed = df.replace('PrivacySuppressed', np.NaN)
    df_nonan = df.dropna(subset=columns)
    return df_nonan

In [ ]:
def quick_add_feature(df):
    df['Public_or_Private'] = df.CONTROL.map({2:0, 1:1, 3:1})

In [ ]:
sheets[2017].info()

## Make Logistic Model

In [ ]:
def create_logistic_regression(df, predictor_columns, predicted_column):
    """
    Creates a logistic regression from input column names to predictor column
    
    Parameters
    ----------
    df: pandas dataframe
        the dataframe's columns should include the predictor_columns and predicted_column
    
    predictor_columns: list
        Should be a subset of columns from df. 
        Should have empty intersection with predicted_column
    
    predicted_column: string
        Should be an element in the list of columns from df. 
        Should not be included in predictor_columns
        
    Returns
    -------
    logreg: logistic regression already trained on training data from predictor columns
    
    Note: This function will change the given dataframe
    """
    
    # Create dataframes( or series) with predictors and  predicted values
    X = df[predictor_columns]
    y = df[predicted_column]
    
    # Scale the data using Robust Scaler
    scale = RobustScaler()
    transformed = scale.fit_transform(X)
    X = pd.DataFrame(transformed, columns = X.columns)
    
    # Create Train and Test Data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    
    # Create a logistic regression model
    logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='lbfgs')
    
    # Fit the model to the training data
    try:
        model_log = logreg.fit(X_train, y_train)
    except:
        model_log = logreg.fit(np.array(X_train).reshape(-1,1), y_train)
        
    # Add new columns to the given data frame with predicted values and probability of correct predictions
    try:
        df['Predicted_'+predicted_column] = logreg.predict(X)
        df['ProbCorrect_Predicted_'+predicted_column] = logreg.predict_proba(X)[:,1]
    except:
        df['Predicted_'+predicted_column] = logreg.predict(np.array(X).reshape(-1,1))
        df['ProbCorrect_Predicted_'+predicted_column] = logreg.predict_proba(np.array(X).reshape(-1,1))[:,1]
    
    return logreg

In [ ]:
# Test create_logistic_regression

create_logistic_regression(df, ['CDR3', 'HI_INC_DEBT_N'], 'Public_or_Private'
                          ).score(df[['CDR3', 'HI_INC_DEBT_N']], df['Public_or_Private'])